# Sentence-BERT Subject  based finetuning

**Model: `sentence-transformers/all-MiniLM-L6-v2`**

**Purpose: Text (cosine) similarity**

In [1]:
%run ../supportvectors-common.ipynb


<div style="color:#aaa;font-size:8pt">
<hr/>
&copy; SupportVectors. All rights reserved. <blockquote>This notebook is the intellectual property of SupportVectors, and part of its training material. 
Only the participants in SupportVectors workshops are allowed to study the notebooks for educational purposes currently, but is prohibited from copying or using it for any other purposes without written permission.

<b> These notebooks are chapters and sections from Asif Qamar's textbook that he is writing on Data Science. So we request you to not circulate the material to others.</b>
 </blockquote>
 <hr/>
</div>



In [2]:
import torch
from sentence_transformers import SentenceTransformer
from svlearn.config.configuration import ConfigurationMixin
from svlearn.encoder_models.sbert_subjects_full_ft import convert_to_pair_dataset, sampled_dataset, get_evaluator
from svlearn.util.hf_text_util import get_train_test_lists, tuples_list_to_dataset

2024-11-03 09:24:24.353762: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-03 09:24:24.374644: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-03 09:24:24.381062: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-03 09:24:24.398915: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-03 09:24:25.369116: W tensorflow/compiler/tf2

In [3]:
# Get the CommentedMap of config (contains paths for data and results directories)
config = ConfigurationMixin().load_config()

/home/chandar/fine-tuning/docs/notebooks/encoder_models


## Load the Model and Dataset from HuggingFace

In [4]:

device = 'cuda' if torch.cuda.is_available() else 'cpu'
# Get the base sentence transformer model
model_name = "sentence-transformers/all-MiniLM-L6-v2"
model = SentenceTransformer(model_name).to(device)

In [5]:

sentence1 = "The rate of change of displacement is velocity"
sentence2 = "Kidney plays an important role in purifying blood"
sentence3 = "Many countries obtained their freedom by 1950"
sentence4 = "Force is proportional to mass"
sentence5 = "Vaccines train our immune system to create antibodies"
sentence6 = "World war 2 was a global conflict between two coalitions - the allies and the axis powers"

sentences = [sentence1, sentence4, sentence2, sentence5, sentence3, sentence6]


In [6]:
embeddings = model.encode(sentences)
similarities = model.similarity(embeddings, embeddings)
print(similarities)


tensor([[ 1.0000,  0.3542, -0.0164,  0.0284,  0.0843, -0.0478],
        [ 0.3542,  1.0000, -0.0098,  0.1015,  0.0422,  0.1267],
        [-0.0164, -0.0098,  1.0000,  0.1488, -0.0401, -0.0213],
        [ 0.0284,  0.1015,  0.1488,  1.0000,  0.0531,  0.0239],
        [ 0.0843,  0.0422, -0.0401,  0.0531,  1.0000,  0.1837],
        [-0.0478,  0.1267, -0.0213,  0.0239,  0.1837,  1.0000]])


Create the evaluator to evaluate the model before and after training

In [7]:
# pick chunks labeled with subjects (biology, physics, history assigned to labels 0, 1, 2 respectively)
_, test = get_train_test_lists(cfg=config)

# Convert to Dataset format
test_dataset = tuples_list_to_dataset(test)

# Sample to max of 500 per label so that the paired dataset is having max of 1500*1499/2
test_dataset = sampled_dataset(test_dataset)

# Create the paired dataset consisting of (sentence1, sentence2, score) from the text/label dataset
test_dataset = convert_to_pair_dataset(test_dataset)

binary_acc_evaluator = get_evaluator(test_dataset=test_dataset)

Filter:   0%|          | 0/9333 [00:00<?, ? examples/s]

Filter:   0%|          | 0/9333 [00:00<?, ? examples/s]

Filter:   0%|          | 0/9333 [00:00<?, ? examples/s]

Evaluate model before training

In [8]:

results = binary_acc_evaluator(model)
results

{'cosine_accuracy': 0.8064,
 'cosine_accuracy_threshold': 0.12211808562278748,
 'cosine_f1': 0.6848559166155733,
 'cosine_f1_threshold': 0.09713000059127808,
 'cosine_precision': 0.6907854050711194,
 'cosine_recall': 0.6790273556231003,
 'cosine_ap': 0.768099852927515,
 'dot_accuracy': 0.8064,
 'dot_accuracy_threshold': 0.12211807072162628,
 'dot_f1': 0.6848559166155733,
 'dot_f1_threshold': 0.09712999314069748,
 'dot_precision': 0.6907854050711194,
 'dot_recall': 0.6790273556231003,
 'dot_ap': 0.7680998529275151,
 'manhattan_accuracy': 0.8002,
 'manhattan_accuracy_threshold': 20.343429565429688,
 'manhattan_f1': 0.6668771708241238,
 'manhattan_f1_threshold': 20.812236785888672,
 'manhattan_precision': 0.6938239159001314,
 'manhattan_recall': 0.6419452887537994,
 'manhattan_ap': 0.7550996576920808,
 'euclidean_accuracy': 0.8064,
 'euclidean_accuracy_threshold': 1.3250523805618286,
 'euclidean_f1': 0.6848559166155733,
 'euclidean_f1_threshold': 1.343778133392334,
 'euclidean_precision':

## Load the fine-tuned model and repeat

In [9]:
config = ConfigurationMixin().load_config()
results_dir = config["paths"]["results"]
# Update Checkpoint Folder
finetuned_model_dir = f'{results_dir}/subject-based-encoder/checkpoint-2000'   
# Load the model
model = SentenceTransformer(finetuned_model_dir)

embeddings = model.encode(sentences)
similarities = model.similarity(embeddings, embeddings)
print(similarities)

/home/chandar/fine-tuning/docs/notebooks/encoder_models
tensor([[ 1.0000,  0.4692, -0.2819, -0.2451, -0.0771, -0.1496],
        [ 0.4692,  1.0000, -0.1757, -0.1116, -0.0724, -0.0366],
        [-0.2819, -0.1757,  1.0000,  0.4889, -0.1105, -0.1134],
        [-0.2451, -0.1116,  0.4889,  1.0000, -0.1291, -0.1757],
        [-0.0771, -0.0724, -0.1105, -0.1291,  1.0000,  0.5889],
        [-0.1496, -0.0366, -0.1134, -0.1757,  0.5889,  1.0000]])


Evaluate model after training

In [10]:

results = binary_acc_evaluator(model)
results

{'cosine_accuracy': 0.9818,
 'cosine_accuracy_threshold': 0.24900028109550476,
 'cosine_f1': 0.972549019607843,
 'cosine_f1_threshold': 0.15513762831687927,
 'cosine_precision': 0.9652694610778443,
 'cosine_recall': 0.9799392097264438,
 'cosine_ap': 0.9960457719069957,
 'dot_accuracy': 0.9818,
 'dot_accuracy_threshold': 0.24900034070014954,
 'dot_f1': 0.972549019607843,
 'dot_f1_threshold': 0.15513765811920166,
 'dot_precision': 0.9652694610778443,
 'dot_recall': 0.9799392097264438,
 'dot_ap': 0.9960457719069957,
 'manhattan_accuracy': 0.9822,
 'manhattan_accuracy_threshold': 19.82466697692871,
 'manhattan_f1': 0.9731523378582202,
 'manhattan_f1_threshold': 20.397037506103516,
 'manhattan_precision': 0.9658682634730539,
 'manhattan_recall': 0.9805471124620061,
 'manhattan_ap': 0.9959320334580082,
 'euclidean_accuracy': 0.9818,
 'euclidean_accuracy_threshold': 1.2255607843399048,
 'euclidean_f1': 0.972549019607843,
 'euclidean_f1_threshold': 1.2998931407928467,
 'euclidean_precision': 0